In [1]:
import torch
import cv2
from deepsort import YoloDetector
from deep_sort_realtime.deepsort_tracker import DeepSort
import time 
import numpy as np
#from drawline import draw_line

In [2]:
detector=YoloDetector("yolov5-crowd/crowdhuman_yolov5m.pt")

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /home/dafodilrat/.cache/torch/hub/master.zip
YOLOv5 🚀 2023-4-4 Python-3.8.10 torch-1.13.1+cu117 CUDA:0 (NVIDIA GeForce GTX 1650 Ti, 3896MiB)

Fusing layers... 
Model summary: 308 layers, 21041679 parameters, 0 gradients
Adding AutoShape... 


Using Device:  cuda


In [9]:
object_tracker = DeepSort(max_age=5,
                n_init=2,
                nms_max_overlap=1.0,
                max_cosine_distance=0.3,
                nn_budget=None,
                override_track_class=None,
                embedder="mobilenet",
                half=True,
                bgr=True,
                embedder_gpu=True,
                embedder_model_name=None,
                embedder_wts=None,
                polygon=False,
                today=None)

In [20]:
cap = cv2.VideoCapture("cctv.mp4")
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

False

In [22]:
def mean(bbox):

  mx=(bbox[0]+bbox[1])/2
  my=(bbox[2]+bbox[3])/2

  return [mx,my]

In [23]:
prev_mean={}
net_dir={}
while cap.isOpened():

    succes, img = cap.read()
 
    start = time.perf_counter()
    
    results = detector.score_frame(img)
    img, detections = detector.plot_boxes(results, img, height=img.shape[0], width=img.shape[1], confidence=0.1)
        
    tracks = object_tracker.update_tracks(detections, frame=img) # bbs expected to be a list of detections, each in tuples of ( [left,top,w,h], confidence, detection_class )

    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        ltrb = track.to_ltrb()
        
        bbox = ltrb

        if track_id not in list(prev_mean.keys()):
            
            prev_mean[track_id]=mean(bbox)
            net_dir[track_id]=0       

        t_vec=[mean(bbox)[0]-prev_mean[track_id][0],mean(bbox)[1]-prev_mean[track_id][1]]
        
        if abs(t_vec[0])>abs(t_vec[1]):
            net_dir[track_id]=net_dir[track_id]+np.sign(t_vec[0])
        
        else:
            net_dir[track_id]=net_dir[track_id]+np.sign(t_vec[1])

        print(track_id," : ",mean(bbox)," - ",net_dir[track_id])
        if net_dir[track_id] <0:
             
             color=(0,255,0) 
        else :
            color=(0,0,255)

        #color=(0,255,0)
        overlay=img.copy()
        alpha=0.2
        cv2.rectangle(overlay,(int(bbox[0]), int(bbox[1])),(int(bbox[2]), int(bbox[3])),color,-1)
        cv2.addWeighted(overlay, alpha, img, 1 - alpha,0, img)
        cv2.putText(img, "ID: " + str(track_id), (int(bbox[0]), int(bbox[1] - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.9,color, 2)
        cv2.rectangle(img,(int(bbox[0]), int(bbox[1])),(int(bbox[2]), int(bbox[3])),color,1)
        cv2.line(img, (299,604),(933,488),(255,0,0), 2)
        m=
        -488-604/933-299
        
        
    end = time.perf_counter()
    totalTime = end - start
    fps = 1 / totalTime


    cv2.putText(img, f'FPS: {int(fps)}', (20,70), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)
    cv2.imshow('img',img)


    if cv2.waitKey(0) & 0xFF == 27: 
        cap.release()
        cv2.destroyAllWindows()


100  :  [529.883948368012, 555.7790466558145]  -  0.0
101  :  [462.9999997615814, 507.0000002384186]  -  0.0
